In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

!pip install prophet

In [2]:
df = pd.read_excel('/content/drive/MyDrive/Solx/Market Pricing (Feb 2023 - Nov 2024).xlsx')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.set_index('Timestamp', inplace=True)
df['Month'] = pd.to_datetime(df['Month'], format='%B %Y')
df = df.sort_values('Month')
df['Average'] = df['Average'].replace(0, np.nan).interpolate(method='linear')
df['Smoothed_Average'] = df['Average'].rolling(window=3, min_periods=1).mean()
full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='H')
df = df.reindex(full_index)
df['Average'] = df['Average'].interpolate(method='linear')
df['Smoothed_Average'] = df['Smoothed_Average'].interpolate(method='linear')



print("Missing Values:")
print(df.isnull().sum())

# Negative price check
negative_prices = df[df['Average'] < 0]
print("Negative Prices:")
print(negative_prices)
print("Total Negative Price Instances:", len(negative_prices))

# Basic stats for price columns
price_stats = df[['Minimum', 'Maximum', 'Average']].describe()
print("Price Stats:", price_stats)

# Monthly Price Stats
monthly_stats = df.groupby('Month')['Average'].agg(['mean', 'std', 'min', 'max'])
print("Monthly Price Stats:", monthly_stats)

# Hourly Price Stats
hourly_stats = df.groupby('Correct Hour')['Average'].agg(['mean', 'std', 'min', 'max'])
print("Hourly Price Stats:", hourly_stats)


#############

# Visualizations
fig = make_subplots(rows=2, cols=2, subplot_titles=('Price Distribution', 'Price Trends', 'Monthly Box Plot', 'Daily Average Prices'))

# PriceDistriHistogram
fig.add_trace(go.Histogram(x=df['Average'], name='Price Distribution', nbinsx=50), row=1, col=1)

# PriceTrendsLPlot
fig.add_trace(go.Scatter(x=df.index, y=df['Average'], name='Price Trend', line=dict(color='blue')), row=1, col=2)

# MonthlyBPlot
fig.add_trace(go.Box(x=df['Month'].dt.strftime('%Y-%m'), y=df['Average'], name='Monthly Price'), row=2, col=1)

# DailyAve PriceTrend
daily_avg = df.groupby(df.index.date)['Average'].mean()
fig.add_trace(go.Scatter(x=daily_avg.index, y=daily_avg.values, name='Daily Average', line=dict(color='green')), row=2, col=2)

# PriceRangeStat
price_range = df['Maximum'] - df['Minimum']
print("Price Range Stats:", price_range.describe())

# for the PriceVolatility
df['Volatility'] = (df['Maximum'] - df['Minimum']) / df['Average']
print("Price Volatility Stats:", df['Volatility'].describe())

# Update layout and show plot
fig.update_layout(height=800, showlegend=True, title='Market Price Analysis')
fig.show()

# Extreme Values Analysis if needed, added just incase for analysis
print("Top 10 Highest Prices:")
print(df.nlargest(10, 'Average')[['Average', 'Month', 'Hour']])

print("Top 10 Lowest Prices:")
print(df.nsmallest(10, 'Average')[['Average', 'Month', 'Hour']])


<ipython-input-2-ba31f95cc66a>:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='H')


Missing Values:
Year                0
Month               0
Date                0
Supply Period       0
Correct Hour        0
Hour                0
Minimum             0
Maximum             0
Average             0
Smoothed_Average    0
dtype: int64
Negative Prices:
                     Year      Month        Date Supply Period  Correct Hour  \
2023-04-06 05:00:00  2023 2023-04-01  2023-04-06    2023-04-30             5   
2023-04-06 07:00:00  2023 2023-04-01  2023-04-06    2023-04-30             7   
2023-04-07 06:00:00  2023 2023-04-01  2023-04-07    2023-04-30             6   
2023-04-07 07:00:00  2023 2023-04-01  2023-04-07    2023-04-30             7   
2023-04-07 08:00:00  2023 2023-04-01  2023-04-07    2023-04-30             8   
...                   ...        ...         ...           ...           ...   
2024-10-24 05:00:00  2024 2024-10-01  2024-10-24    2024-10-31             5   
2024-10-24 06:00:00  2024 2024-10-01  2024-10-24    2024-10-31             6   
2024-11-01 07:

Top 10 Highest Prices:
                       Average      Month  Hour
2024-03-01 19:00:00  99.865904 2024-03-01    20
2024-03-04 21:00:00  76.843119 2024-03-01    22
2024-05-13 14:00:00  64.630246 2024-05-01    15
2024-03-04 19:00:00  49.734615 2024-03-01    20
2024-03-01 21:00:00  48.790913 2024-03-01    22
2024-08-13 20:00:00  47.909181 2024-08-01    21
2024-03-04 18:00:00  47.803788 2024-03-01    19
2024-03-01 18:00:00  47.015293 2024-03-01    19
2023-03-21 16:00:00  45.695099 2023-03-01    17
2024-09-22 19:00:00  44.812456 2024-09-01    20
Top 10 Lowest Prices:
                       Average      Month  Hour
2024-09-22 12:00:00 -24.430888 2024-09-01    13
2024-09-22 11:00:00 -15.104331 2024-09-01    12
2023-04-07 09:00:00  -7.302328 2023-04-01    10
2023-04-07 10:00:00  -7.289506 2023-04-01    11
2024-09-21 22:00:00  -6.900563 2024-09-01    23
2023-04-07 11:00:00  -5.551434 2023-04-01    12
2023-11-27 06:00:00  -5.150980 2023-11-01     7
2023-11-27 05:00:00  -5.130174 2023-11-01  

In [3]:
# Daily Volatility Analysis
daily_stats = df.groupby(df.index.date).agg({
'Average': ['mean', 'std', 'min', 'max'],
'Volatility': 'mean'
}).round(4)
print("Daily Price Stats:")
print(daily_stats.head(10))
fig_volatility = go.Figure()
fig_volatility.add_trace(
go.Scatter(x=daily_stats.index,
y=daily_stats['Volatility']['mean'],
name='Daily Volatility',
line=dict(color='red'))
)
fig_volatility.update_layout(
title='Daily Volatility Over Time',
xaxis_title='Date',
yaxis_title='Volatility',
showlegend=True
)
fig_volatility.show()

Daily Price Stats:
           Average                          Volatility
              mean     std     min      max       mean
2023-02-26  7.0069  2.4509  3.2091  11.8795     0.5942
2023-02-27  3.7544  0.4154  2.7160   4.5248     0.5169
2023-02-28  4.1828  0.7093  2.7053   6.4858     0.6222
2023-03-01  5.3175  1.9491  2.9663  11.1656     0.8215
2023-03-02  4.0954  0.5826  2.9871   5.2561     0.6822
2023-03-03  4.2021  0.8281  2.3168   6.0304     0.6628
2023-03-04  5.0328  1.5343  2.3107   8.3099     0.8455
2023-03-05  4.3085  1.8933  2.3118   8.7755     0.8841
2023-03-06  4.2026  1.1492  1.8910   6.2679     0.9050
2023-03-07  4.6532  1.5270  2.0424   7.0635     0.8521


In [4]:
#Monthly Trends Analysis
monthly_detailed = df.groupby('Month').agg({
'Average': ['mean', 'std', 'min', 'max', 'count'],
'Minimum': 'min',
'Maximum': 'max'
}).round(4)
print("Monthly Stats:")
print(monthly_detailed)
fig_monthly = go.Figure()
fig_monthly.add_trace(
go.Bar(x=monthly_detailed.index,
y=monthly_detailed['Average']['mean'],
name='Monthly Average',
marker_color='blue')
)
fig_monthly.add_trace(
go.Scatter(x=monthly_detailed.index,
y=monthly_detailed['Average']['max'],
name='Price Range (Max)',
line=dict(color='red', dash='dash'))
)
fig_monthly.add_trace(
go.Scatter(x=monthly_detailed.index,
y=monthly_detailed['Average']['min'],
name='Price Range (Min)',
line=dict(color='green', dash='dash'))
)
fig_monthly.update_layout(
title='Monthly Price Trends with Range',
xaxis_title='Month',
yaxis_title='Price',
showlegend=True
)
fig_monthly.show()


Monthly Stats:
           Average                                  Minimum   Maximum
              mean     std      min      max count      min       max
Month                                                                
2023-02-01  4.9814  2.0679   2.7053  11.8795    72   0.0000   16.3602
2023-03-01  5.6622  4.3983   0.2513  45.6951   744  -8.0263  133.0280
2023-04-01  5.9569  2.9137  -7.3023  21.6210   720  -8.1869   29.4218
2023-05-01  6.9025  3.8170   0.5123  28.3909   744  -8.0040   49.8228
2023-06-01  4.5719  2.8487   0.9308  26.7328   720  -8.0079   35.2249
2023-07-01  4.2457  2.8440   0.1826  22.5143   744  -8.0263   31.5420
2023-08-01  4.5116  2.5157   1.5660  20.4761   744   0.0000   31.4457
2023-09-01  4.3843  1.9149   1.3894  14.9559   720   0.0000   30.4391
2023-10-01  4.8534  3.2537   0.1193  28.1388   744  -7.8018  124.7666
2023-11-01  3.2396  2.0661  -5.1510  25.1076   720  -8.3461  186.5161
2023-12-01  3.5111  2.1142  -0.0010  22.7901   744  -7.8186   28.9342
2024-

In [5]:
#Hourly Pattern Analysis
hourly_detailed = df.groupby('Hour').agg({
'Average': ['mean', 'std', 'min', 'max'],
'Minimum': 'min',
'Maximum': 'max'
}).round(4)
print("Hourly Price Patterns:")
print(hourly_detailed)
fig_hourly = go.Figure()
fig_hourly.add_trace(
go.Scatter(x=hourly_detailed.index,
y=hourly_detailed['Average']['mean'],
name='Average Price',
line=dict(color='blue'))
)
fig_hourly.add_trace(
go.Scatter(x=hourly_detailed.index,
y=hourly_detailed['Average']['mean'] + hourly_detailed['Average']['std'],
name='Upper Bound',
line=dict(color='lightblue', dash='dash'))
)
fig_hourly.add_trace(
go.Scatter(x=hourly_detailed.index,
y=hourly_detailed['Average']['mean'] - hourly_detailed['Average']['std'],
name='Lower Bound',
fill='tonexty',
line=dict(color='lightblue', dash='dash'))
)
fig_hourly.update_layout(
title='24-Hour Price Pattern',
xaxis_title='Hour of Day',
yaxis_title='Price',
showlegend=True
)
fig_hourly.show()

Hourly Price Patterns:
     Average                            Minimum   Maximum
        mean     std      min      max      min       max
Hour                                                     
1     4.0398  1.6916  -2.8358  15.4128  -2.8445   56.5801
2     3.6443  1.3054   0.0323  14.9339  -2.8246   32.0423
3     3.3319  1.1348  -1.8486  11.5336  -5.5008   26.8972
4     3.0932  1.0059  -1.5683  10.2869  -8.1193   15.1010
5     3.2600  1.1864  -5.0648  10.3697  -8.1042   39.9603
6     3.0648  1.0991  -5.1302   9.1290  -8.1869   23.2948
7     2.4908  0.9529  -5.1510   8.3337  -8.3519   49.1331
8     2.7609  1.1448  -4.0380   9.6910  -8.3548   37.9375
9     3.4284  2.2341  -2.9231  40.2845  -8.3461   61.9988
10    3.5900  2.1421  -7.3023  33.2051  -8.2527   51.0222
11    4.2141  2.3687  -7.2895  24.7501  -7.9740   41.4019
12    3.8282  2.1733 -15.1043  22.8371 -30.4127   97.3906
13    3.5579  2.2064 -24.4309  18.6891 -30.3407   44.9035
14    5.8140  4.0075  -0.0041  25.9816 -11.3753  

In [7]:
# Price Stability Analysis
df['Price_Range'] = df['Maximum'] - df['Minimum']
df['Coefficient_of_Variation'] = df.groupby('Month')['Average'].transform(lambda x: x.std() / x.mean())
stability_metrics = df.groupby('Month').agg({
    'Price_Range': 'mean',
    'Coefficient_of_Variation': 'mean',
    'Volatility': 'mean'
}).round(4)

print("Monthly Stability Metrics:")
print(stability_metrics)

fig_stability = go.Figure()
for column in stability_metrics.columns:
    fig_stability.add_trace(go.Scatter(x=stability_metrics.index,y=stability_metrics[column],name=column)
    )

fig_stability.update_layout(
    title='Monthly Price Stability',
    xaxis_title='Month',
    yaxis_title='Value',
    showlegend=True
)

fig_stability.show()


Monthly Stability Metrics:
            Price_Range  Coefficient_of_Variation  Volatility
Month                                                        
2023-02-01       2.8067                    0.4151      0.5778
2023-03-01       4.3343                    0.7768      0.7369
2023-04-01       4.7397                    0.4891     -0.2574
2023-05-01       5.5475                    0.5530      0.7831
2023-06-01       2.1976                    0.6231      0.4355
2023-07-01       2.8143                    0.6698      0.6926
2023-08-01       3.4463                    0.5576      0.6288
2023-09-01       2.8829                    0.4368      0.5724
2023-10-01       2.9450                    0.6704      0.5618
2023-11-01       3.7773                    0.6378      0.9499
2023-12-01       3.7517                    0.6021      0.9735
2024-01-01       3.2637                    0.6108      0.8005
2024-02-01       2.7678                    0.8161      0.7171
2024-03-01       4.6459                    

In [8]:
#Correlation Analysis
price_correlation = df[['Minimum', 'Maximum', 'Average']].corr().round(4)
print("Price Component Correlation Matrix:")
print(price_correlation)
fig_corr = go.Figure(data=go.Heatmap(
z=price_correlation,
x=price_correlation.columns,
y=price_correlation.columns,
colorscale='RdBu',
zmin=-1, zmax=1,
text=np.round(price_correlation, 2),
texttemplate='%{text}',
textfont={"size": 12},
showscale=True
))
fig_corr.update_layout(
title='Price Correlation Matrix',
width=600,
height=500
)
fig_corr.show()
time_correlation = df.pivot_table(
values='Average',
index=df['Month'].dt.strftime('%Y-%m'),
columns='Hour',
aggfunc='mean'
).round(4)
print("Hour-Month Price Correlation:")
print(time_correlation)


Price Component Correlation Matrix:
         Minimum  Maximum  Average
Minimum   1.0000   0.4579   0.7587
Maximum   0.4579   1.0000   0.8129
Average   0.7587   0.8129   1.0000


Hour-Month Price Correlation:
Hour         1       2       3       4       5       6       7       8   \
Month                                                                     
2023-02  4.5959  3.9007  3.9097  3.7930  4.1070  5.5406  5.4703  4.3423   
2023-03  4.9883  4.5290  4.0893  4.0079  4.6253  4.4974  3.8018  3.1788   
2023-04  5.6707  5.0977  4.4431  4.1235  4.5641  4.1316  2.7910  3.5181   
2023-05  5.2240  5.2147  4.9251  4.7212  5.1917  4.2207  3.2986  4.7430   
2023-06  4.0741  3.9230  3.7751  3.6325  3.6124  3.3614  2.8462  3.2057   
2023-07  4.1877  3.5535  3.1507  2.9396  3.0366  2.5426  2.2598  2.7845   
2023-08  4.1547  3.8644  3.4271  3.2634  3.2084  2.9818  2.5751  2.9337   
2023-09  4.1385  3.6802  3.6319  3.1892  3.3567  3.2645  2.5648  2.9538   
2023-10  3.9939  3.7850  3.5321  3.4143  3.5809  3.4265  2.8664  3.5402   
2023-11  2.8008  2.6939  2.5779  2.3002  2.2414  2.2653  1.6671  1.9586   
2023-12  3.0654  2.7297  2.6509  2.4961  2.4528  2.6229  2.2601  2.256